# Setup

In [1]:
import os
from pathlib import Path
import sys

# If we're using Google Colab, we set the environment variable to point to the relevant folder in our Google Drive:
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
    os.environ['SKIN_LESION_CLASSIFICATION'] = '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification'

# Otherwise, we use the environment variable on our local system:
project_environment_variable = "SKIN_LESION_CLASSIFICATION"

# Path to the root directory of the project:
project_path = Path(os.environ.get(project_environment_variable))

# Relative path to /scripts (from where custom modules will be imported):
scripts_path = project_path.joinpath("scripts")

# Add this path to sys.path so that Python will look there for modules:
sys.path.append(str(scripts_path))

# Now import path_step from our custom utils module to create a dictionary to all subdirectories in our root directory:
from utils import path_setup
path = path_setup.subfolders(project_path)

Mounted at /content/drive
path['project'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification
path['models'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models
path['notebooks'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/notebooks
path['scripts'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/scripts
path['images'] : /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/images


<a id='model_setup'></a>
# Model setup

In [2]:
from typing import Type, Union      # For type hints
from processing import process      # Custom module for processing metadata

data_dir: Path = path["images"]     # Path to directory containing metadata.csv file
csv_filename: str = "metadata.csv"  # The filename

tvr: int = 3              # Ratio of training set to validation set. See discussion below for explanation.
seed: int = 0             # Random seed for parts of the process where randomness is called for.
keep_first: bool = False  # If False, then, for each lesion, we choose a random image to assign to our training set.
stratified: bool = True   # If True, we stratify classes so that the proportions remain as stable as possible after train/val split.
                          # If False, the proportions will be roughly similar.

to_classify: Union[list, dict] = ["mel",   # These are the lesion types we are interested in classifying.
                                  "bcc",   # Any missing ones will be grouped together as the 0-label class: no need to write "other" here.
                                  "akiec", # If 'other' is not desired, use restrict_to attribute above
                                  "nv",]   # Can also be a dictionary, like { 'malignant' : ['mel', 'bcc'], 'benign' : ['nv', 'bkl']}

train_one_img_per_lesion: Union[None, bool] = True

sample_size: Union[None, dict] = {"mel": 2000,     # Handling class imbalance by upsampling minority classes/downsampling majority classes
                                  "bcc": 2000,     # Specify how many images of each lesion diagnosis we want in our training set.
                                  "akiec": 2000,
                                  "nv": 2000,
                                  "other" : 2000,} # Could also leave out "other" here, and include e.g. "df: 2000" if we wanted to.

In [3]:
# Create an instance of the process class with attribute values as above.
balance_t1 = process(data_dir=data_dir,
                  csv_filename=csv_filename,
                  tvr=tvr,
                  seed=seed,
                  keep_first=keep_first,
                  stratified=stratified,
                  to_classify=to_classify,
                  train_one_img_per_lesion=train_one_img_per_lesion,
                  sample_size=sample_size,)

- Loaded file '/content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/images/metadata.csv'.
- Inserted 'num_images' column in dataframe, to the right of 'lesion_id' column.
- Inserted 'label' column in dataframe, to the right of 'dx' column: 
  {'bkl': 0, 'df': 0, 'vasc': 0, 'akiec': 1, 'bcc': 2, 'mel': 3, 'nv': 4}
- Added 'set' column to dataframe, with values 't1', 'v1', 'ta', and 'va', to the right of 'localization' column.
- Basic, overall dataframe (pre-train/test split): self.df
- Balancing classes in training set.
- Balanced training set (one image per lesion): self.df_train
- Validation set (not expanded, one image per lesion): self.df_val1
- Validation set (not expanded, use all images of each lesion): self.df_val_a
- Small sample dataframes for code testing: self._df_train_code_test, self._df_val1_code_test, self._df_val_a_code_test


In [4]:
import torchvision.transforms as transforms
transform = transforms.Compose([
transforms.Resize((224,224)), # Resize images to fit ResNet input size
transforms.ToTensor(),
transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize with ImageNet stats
])

In [5]:
import pandas as pd
from typing import Union, List, Callable
import torchvision.models as models

source: Union[process, pd.DataFrame] = balance_t1        # Processed data to be fed into model for training.
                                                      # Must either be an instance of the process class, or a dataframe of the same format as source.df if source were an instance of the process class.
model_dir: Path = path["models"]                      # Path to directory where models/model info/model results are stored.

transform: Union[None,
                 transforms.Compose,
                 List[Callable]] = transform     # Transform to be applied to images before feeding into neural network.

filename_stem: Union[None, str] = "rn18"         # For saving model and related files. Default "rn18" (if ResNet model) or "EffNet" (if EfficientNet), or "cnn".
filename_suffix: Union[None, str] = "notfm" # Something descriptive and unique for future reference. Default empty string "".

# model: Union[None, models.ResNet, models.EfficientNet] = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT) # Pre-trained model. Default: ResNet18.
model: Union[None, models.ResNet, models.EfficientNet] = models.resnet18(weights="ResNet18_Weights.DEFAULT")
unfreeze_last: Union[None, bool] = True
# overwrite: Union[None, bool] = True

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 176MB/s]


In [6]:
# Create an instance of the resnet18 class with attribute values as above.
from multiclass_models import cnn

rn18_balance_t1 = cnn(source=source,
                    model_dir=model_dir,
                    transform=transform,
                    filename_stem=filename_stem,
                    filename_suffix=filename_suffix,
                    model=model,
                    unfreeze_last=unfreeze_last,)

New files will be created. 
Base filename: rn18_t1_bal_uflast_10e_notfm_00
Attributes saved to file: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_t1_bal_uflast_10e_notfm_00_attributes.json


In [7]:
import time

tic = time.time()
rn18_balance_t1.train()
toc = time.time()
print(f"Elapsed time: {toc - tic}")

Batch: 0, running loss: 1.5573744773864746
Batch: 10, running loss: 16.673335075378418
Batch: 20, running loss: 28.54046380519867
Batch: 30, running loss: 40.669339179992676
Batch: 40, running loss: 50.65391480922699
Batch: 50, running loss: 59.9860680103302
Batch: 60, running loss: 69.37498563528061
Batch: 70, running loss: 78.08221799135208
Batch: 80, running loss: 87.61789280176163
Batch: 90, running loss: 96.28082758188248
Batch: 100, running loss: 104.78865140676498
Batch: 110, running loss: 112.67262667417526
Batch: 120, running loss: 119.94523084163666
Batch: 130, running loss: 127.91914927959442
Batch: 140, running loss: 134.404865026474
Batch: 150, running loss: 140.58478459715843
Batch: 160, running loss: 148.41836887598038
Batch: 170, running loss: 155.75965869426727
Batch: 180, running loss: 161.5983640551567
Batch: 190, running loss: 167.86764460802078
Batch: 200, running loss: 175.49469089508057
Batch: 210, running loss: 182.28068128228188
Batch: 220, running loss: 189.44

In [ ]:
# import torch
# import torch.nn as nn

# instance = rn18_balance_t1

# model = models.resnet18()
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, len(instance.label_codes))

# file_path_pth = instance.model_dir.joinpath(instance._filename + ".pth")
# state_dict = torch.load(file_path_pth)
# model.load_state_dict(state_dict)

# # model = models.efficientnet_b0()
# # num_ftrs = model.classifier[1].in_features
# # model.classifier[1] = nn.Linear(num_ftrs, len(instance.label_codes))

# instance.model = model
# instance.state_dict = state_dict

In [8]:
# from utils import print_header
from multiclass_models import get_probabilities

instance = rn18_balance_t1

tic = time.time()

instance.df_probabilities_val1 = get_probabilities(df=instance.df_val1,
                                                   data_dir=instance.data_dir,
                                                   model_dir=instance.model_dir,
                                                   model=instance.model,
                                                   filename=instance._filename,
                                                   label_codes=instance.label_codes,
                                                   transform=instance.transform,
                                                   batch_size=instance.batch_size,
                                                   Print=False,
                                                   save_as=instance._filename + "_val1",)

toc = time.time()

print(f"Elapsed time: {toc - tic}")

instance.df_probabilities_val_a = get_probabilities(df=instance.df_val_a,
                                                    data_dir=instance.data_dir,
                                                    model_dir=instance.model_dir,
                                                    model=instance.model,
                                                    filename=instance._filename,
                                                    label_codes=instance.label_codes,
                                                    transform=instance.transform,
                                                    batch_size=instance.batch_size,
                                                    Print=False,
                                                    save_as=instance._filename + "_val_a",)
tic = time.time()

print(f"Elapsed time: {tic - toc}")

Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_t1_bal_uflast_10e_notfm_00_val1_probabilities.csv
Elapsed time: 26.631510734558105
Saving probabilities: /content/drive/MyDrive/Colab Notebooks/skin-lesion-classification/models/rn18_t1_bal_uflast_10e_notfm_00_val_a_probabilities.csv
Elapsed time: 35.97200417518616


In [ ]:
# instance = rn18_base_t1

# file_path1 = instance.model_dir.joinpath("rn18_t1_bal_uflast_10e_notfm_00_val1_probabilities.csv")
# file_path_a = instance.model_dir.joinpath("rn18_t1_bal_uflast_10e_notfm_00_val_a_probabilities.csv")

# instance.df_probabilities_val1 = pd.read_csv(file_path1, index_col=0)
# instance.df_probabilities_val_a = pd.read_csv(file_path_a, index_col=0)

In [9]:
from utils import print_header

instance = rn18_balance_t1

print_header("Validation set: one image per lesion")
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')]
display(instance.df_probabilities_val1[display_columns].head())

print_header("Validation set: all images per lesion")
display(instance.df_probabilities_val_a[display_columns].head())


VALIDATION SET: ONE IMAGE PER LESION



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv
0,HAM_0002730,ISIC_0025661,bkl,0.000862,8.549663e-01,1.417131e-01,2.421762e-05,0.002435
1,HAM_0001466,ISIC_0027850,bkl,0.045447,7.546793e-03,9.458811e-01,9.090114e-04,0.000216
2,HAM_0002761,ISIC_0029068,bkl,0.220750,9.414385e-02,6.730878e-01,9.995591e-06,0.012008
3,HAM_0004234,ISIC_0029396,bkl,0.000019,6.780808e-05,1.232119e-03,7.890170e-03,0.990791
4,HAM_0001949,ISIC_0025767,bkl,0.999964,1.919044e-07,7.892933e-07,8.280515e-07,0.000034



VALIDATION SET: ALL IMAGES PER LESION



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv
0,HAM_0002730,ISIC_0026769,bkl,0.000029,0.000303,0.999664,3.544990e-07,3.576648e-06
1,HAM_0002730,ISIC_0025661,bkl,0.000862,0.854966,0.141713,2.421762e-05,2.434572e-03
2,HAM_0001466,ISIC_0031633,bkl,0.340047,0.010757,0.639567,4.271342e-03,5.358230e-03
3,HAM_0001466,ISIC_0027850,bkl,0.045447,0.007547,0.945881,9.090114e-04,2.160815e-04
4,HAM_0002761,ISIC_0029176,bkl,0.000170,0.002487,0.997342,1.752473e-09,8.264573e-07


In [11]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = rn18_balance_t1

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = None#{ 'max' : ['mel', 'bcc', 'akiec'], 'min' : ['nv'], 'mean' : ['other']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = None# OrderedDict([('mel',0.4), ('bcc', 0.4), ('akiec', 0.4)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = None#OrderedDict([('nv',0.6)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = None #OrderedDict([('mel',1), ('bcc',1), ('akiec',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')] + ['pred', 'pred_final']
display(instance.df_pred_val1[display_columns].head())

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a[display_columns].head())


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0025661,bkl,0.000862,8.549663e-01,1.417131e-01,2.421762e-05,0.002435,1,1
1,HAM_0001466,ISIC_0027850,bkl,0.045447,7.546793e-03,9.458811e-01,9.090114e-04,0.000216,2,2
2,HAM_0002761,ISIC_0029068,bkl,0.220750,9.414385e-02,6.730878e-01,9.995591e-06,0.012008,2,2
3,HAM_0004234,ISIC_0029396,bkl,0.000019,6.780808e-05,1.232119e-03,7.890170e-03,0.990791,4,4
4,HAM_0001949,ISIC_0025767,bkl,0.999964,1.919044e-07,7.892933e-07,8.280515e-07,0.000034,0,0



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0026769,bkl,0.000029,0.000303,0.999664,3.544990e-07,3.576648e-06,2,1
1,HAM_0002730,ISIC_0025661,bkl,0.000862,0.854966,0.141713,2.421762e-05,2.434572e-03,1,1
2,HAM_0001466,ISIC_0031633,bkl,0.340047,0.010757,0.639567,4.271342e-03,5.358230e-03,2,2
3,HAM_0001466,ISIC_0027850,bkl,0.045447,0.007547,0.945881,9.090114e-04,2.160815e-04,2,2
4,HAM_0002761,ISIC_0029176,bkl,0.000170,0.002487,0.997342,1.752473e-09,8.264573e-07,2,2


In [12]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = rn18_balance_t1
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 2
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,98,4,33,11,79,225,0.435556
akiec,13,13,23,3,5,57,0.22807
bcc,9,2,64,0,7,82,0.780488
mel,13,2,14,63,62,154,0.409091
nv,35,3,102,33,"1,178","1,351",0.871947
All,168,24,236,110,"1,331","1,869",_
precision,0.583333,0.541667,0.271186,0.572727,0.885049,_,0.518507



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,116,2,27,12,68,225,0.515556
akiec,17,11,24,1,4,57,0.192982
bcc,10,2,66,0,4,82,0.804878
mel,17,3,21,62,51,154,0.402597
nv,41,6,113,39,"1,152","1,351",0.852702
All,201,24,251,114,"1,279","1,869",_
precision,0.577114,0.458333,0.262948,0.54386,0.900704,_,0.520308


In [13]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = rn18_balance_t1

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 2
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.757624,0.54503,NaN,NaN,0.539139,0.515591,0.518507,0.481777,0.891457,0.88959,0.84957



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.752809,0.553743,NaN,NaN,0.525174,0.510267,0.520308,0.492139,0.89114,0.893283,0.84759


In [14]:
from collections import OrderedDict
from typing import Dict, List
from multiclass_models import final_prediction

instance = rn18_balance_t1

raw_probabilities_df1: pd.DataFrame = instance.df_probabilities_val1
raw_probabilities_df_a: pd.DataFrame = instance.df_probabilities_val_a
aggregate_method: Union[None, Dict[str, List[str]]] = { 'max' : ['mel', 'bcc', 'akiec'], 'min' : ['nv'], 'mean' : ['other']}
threshold_dict_help: Union[None, OrderedDict[str, float]] = OrderedDict([('mel',0.4), ('bcc', 0.4), ('akiec', 0.4)])
threshold_dict_hinder: Union[None, OrderedDict[str, float]] = None#OrderedDict([('nv',0.6)])
votes_to_win_dict: Union[None, OrderedDict[str, int]] = None #OrderedDict([('mel',1), ('bcc',1), ('akiec',1)])
label_codes: Dict[int, str] = instance.label_codes
prefix: Union[None, str] = 'prob_'

print_header("Validation set, one image per lesion: combining probabilities and making predictions")

instance.df_pred_val1 = final_prediction(raw_probabilities_df=raw_probabilities_df1,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)
display_columns = ['lesion_id', 'image_id', 'dx'] + [col for col in instance.df_probabilities_val1.columns if col.startswith('prob')] + ['pred', 'pred_final']
display(instance.df_pred_val1[display_columns].head())

print_header("Validation set, all images per lesion: combining probabilities, making predictions, and combining predictions")

instance.df_pred_val_a = final_prediction(raw_probabilities_df=raw_probabilities_df_a,
                                          threshold_dict_help=threshold_dict_help,
                                          threshold_dict_hinder=threshold_dict_hinder,
                                          votes_to_win_dict=votes_to_win_dict,
                                          label_codes=label_codes,)

display(instance.df_pred_val_a[display_columns].head())


VALIDATION SET, ONE IMAGE PER LESION: COMBINING PROBABILITIES AND MAKING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0025661,bkl,0.000862,8.549663e-01,1.417131e-01,2.421762e-05,0.002435,1,1
1,HAM_0001466,ISIC_0027850,bkl,0.045447,7.546793e-03,9.458811e-01,9.090114e-04,0.000216,2,2
2,HAM_0002761,ISIC_0029068,bkl,0.220750,9.414385e-02,6.730878e-01,9.995591e-06,0.012008,2,2
3,HAM_0004234,ISIC_0029396,bkl,0.000019,6.780808e-05,1.232119e-03,7.890170e-03,0.990791,4,4
4,HAM_0001949,ISIC_0025767,bkl,0.999964,1.919044e-07,7.892933e-07,8.280515e-07,0.000034,0,0



VALIDATION SET, ALL IMAGES PER LESION: COMBINING PROBABILITIES, MAKING PREDICTIONS, AND COMBINING PREDICTIONS



,lesion_id,image_id,dx,prob_other,prob_akiec,prob_bcc,prob_mel,prob_nv,pred,pred_final
0,HAM_0002730,ISIC_0026769,bkl,0.000029,0.000303,0.999664,3.544990e-07,3.576648e-06,2,1
1,HAM_0002730,ISIC_0025661,bkl,0.000862,0.854966,0.141713,2.421762e-05,2.434572e-03,1,1
2,HAM_0001466,ISIC_0031633,bkl,0.340047,0.010757,0.639567,4.271342e-03,5.358230e-03,2,2
3,HAM_0001466,ISIC_0027850,bkl,0.045447,0.007547,0.945881,9.090114e-04,2.160815e-04,2,2
4,HAM_0002761,ISIC_0029176,bkl,0.000170,0.002487,0.997342,1.752473e-09,8.264573e-07,2,2


In [15]:
# from utils import print_header
from evaluation import weighted_average_f, confusion_matrix_with_metric

instance = rn18_balance_t1
map_labels = instance.label_codes

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']

txp1 = pd.crosstab(target1,prediction1,margins=True,dropna=False)
txp_a = pd.crosstab(target_a,prediction_a,margins=True,dropna=False)

beta = 2
# Weights inversely proportional to relative class size in the training set, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

instance.cm1 = confusion_matrix_with_metric(AxB=txp1,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

instance.cm_a = confusion_matrix_with_metric(AxB=txp_a,
                                            lst=None,
                                            full_pad=True,
                                            func=weighted_average_f,
                                            beta=beta,
                                            weights=weights,
                                            percentage=False,
                                            map_labels=map_labels)

print_header("Confusion matrix: validation set, one image per lesion")
display(instance.cm1.fillna('_'))

print_header("Confusion matrix: validation set, all images per lesion")
display(instance.cm_a.fillna('_'))


CONFUSION MATRIX: VALIDATION SET, ONE IMAGE PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,96,4,35,13,77,225,0.426667
akiec,13,12,24,3,5,57,0.210526
bcc,9,2,64,0,7,82,0.780488
mel,12,2,15,69,56,154,0.448052
nv,34,3,107,38,"1,169","1,351",0.865285
All,164,23,245,123,"1,314","1,869",_
precision,0.585366,0.521739,0.261224,0.560976,0.88965,_,0.517107



CONFUSION MATRIX: VALIDATION SET, ALL IMAGES PER LESION



predicted,other,akiec,bcc,mel,nv,All,recall
actual,,,,,,,
other,114,3,28,14,66,225,0.506667
akiec,17,10,25,1,4,57,0.175439
bcc,10,1,67,0,4,82,0.817073
mel,16,3,22,67,46,154,0.435065
nv,41,6,118,44,"1,142","1,351",0.8453
All,198,23,260,126,"1,262","1,869",_
precision,0.575758,0.434783,0.257692,0.531746,0.904913,_,0.519225


In [16]:
# from utils import print_header
from evaluation import metric_dictionary
# import pandas as pd

instance = rn18_balance_t1

target1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['label']
prediction1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id')['pred_final']
probabilities1 = instance.df_probabilities_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities1 = instance.df_pred_val1.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

target_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['label']
prediction_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id')['pred_final']
probabilities_a = instance.df_probabilities_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')
agg_probabilities_a = instance.df_pred_val_a.drop_duplicates(subset='lesion_id').filter(regex=r'^prob_')

beta = 2
# Weights inversely proportional to relative class size, giving more importance to smaller classes.
weights = 1/instance.df_train['label'].value_counts(normalize=True).sort_index().values # None

print_header("Baseline model: other metrics")

instance.metric_dict1 = metric_dictionary(target=target1,
                                          prediction=prediction1,
                                          probabilities=probabilities1)

instance.metric_dict_a = metric_dictionary(target=target_a,
                                          prediction=prediction_a,
                                          probabilities=probabilities_a)

print("\nOne image per lesion".upper())
display(pd.DataFrame(instance.metric_dict1))

print("\nAll images per lesion".upper())
display(pd.DataFrame(instance.metric_dict_a))


BASELINE MODEL: OTHER METRICS


ONE IMAGE PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.754414,0.546204,NaN,NaN,0.533512,0.512101,0.517107,0.482242,0.891457,0.88959,0.84957



ALL IMAGES PER LESION


,ACC,BACC,precision,recall,F1/2,F1,F2,MCC,ROC-AUC mac,ROC-AUC wt,ROC-AUC wt*
0,0.749064,0.555909,NaN,NaN,0.519212,0.506696,0.519225,0.491781,0.89114,0.893283,0.84759
